In [2]:
import torch

### Core concept:
- `tensor.view()` creates a new view of the same underlying data but with a different shape
- Memory-efficient and no new data is allocated.
- Shared memory: Because the original tensor and its view share the same memory, any in-place changes to the view will also modify the original tensor, and vice versa. 

In [4]:
x  = torch.arange(12)
print(x.is_contiguous())

True


### `.view()` works only on <b>CONTIGUOUS TENSORS</b>
- If the tensor is not contiguous, `.view()` will throw an error

In [ ]:
x = torch.randn(2, 3)
y = x.t()
print(y.is_contiguous())  # False
y.view(6)                 # RuntimeError


y = y.contiguous().view(6)  # ✅ Works now

### CONTIGUITY in TENSORS
- Logical order of the tensor is the same as the physical order in memory
Eg. ` a = [[1,2,3],[4,5,6]]
- In memory, it looks like : [1,2,3,4,5,6]
- stride = `(s_x,s_y)` = `(3,1)` (number of places moved in memory to reach the next row element,  number of places moved in memory to reach the next col element)


#### For a tensor with shape (s_0, s_1, ..., s_n-1), the strides (t_0, t_1, ..., t_n-1) must satisfy the following: Innermost dimension stride: 
- The stride of the last dimension (\(t_{n-1}\)) must be 1. 
- This means adjacent elements in the final dimension are also adjacent in memory.
- General stride condition: For any other dimension i, the stride t_i must be equal to the product of the sizes of all dimensions to its right.

Eg Consider a tensor x with shape (4, 3, 2). 
- Its expected contiguous strides would be (6, 2, 1).
- `t_2 = 1 `;  `t_1 = t_2 * s_1` ; `t_0 = t_1 * s_0`

### Operations that destroy CONTIGUITY:
- `.permute`
- `.transpose` 

In [ ]:
# Row/Col matrices are contiguous even when transposed

y = torch.rand((1,6))
print(y.is_contiguous())
print(y.T.is_contiguous())
print(y.reshape(2,3).is_contiguous())

print("- - - - - -- - - - - -- - - -- - - -")
x = torch.tensor(([1,2,3],[4,5,6]))
print(x.is_contiguous())
print(x.T.is_contiguous())
print(x.reshape(3,2).is_contiguous())

True
True
True
- - - - - -- - - - - -- - - -- - - -
True
False
True


### The `-1` magic dimension
- In .view() you can enter -1 to let PyTorch infer the dimension of the reshaped tensor automatically 

- PyTorch will compute the missing dimension accordingly whichever position `-1` is in

- Only one -1 is allowed.

- The total number of elements must match.

In [18]:
a = torch.arange(12)
b = a.view(3,-1)

### WHen to use `.view()` vs `.reshape()`
- When you know that the tensor is contiguous, then use `.view()`
- If not sure, use `.reshape()`

But `.reshape()` is less memory efficient than `.view()` because it creates a new tensor in memory, although its not guaranteed to do so
- `.reshape()` creates a copy and allocates new memory when the original tensor is not contiguous. A tensor becomes non-contiguous after certain operations, like `.permute()` or `.transpose()`, which reorder the dimensions without changing the underlying data layout. 

- If the input tensor is already contiguous, reshape() will simply return a view of the original tensor, just like view() does. In this case, no new memory is allocated, and the operation is just as memory-efficient as view()

- `.view()` is more restrictive and will only return a view if the tensor is contiguous. If the tensor is non-contiguous, calling `.view()` will result in a RuntimeError. This forces the user to explicitly handle the contiguity, typically by calling `.contiguous()` first, which performs the memory copy manually.

In [ ]:
a = torch.rand((2,3))
print(a.is_contiguous())
b = a.T
print(b.is_contiguous())


a.view(3,-1) # Will work because the tensor is contiguos
b.view(2,-1) # runtime error because b is not contiguous

print(b.reshape(2,-1)) # Will work even when the tensor is not contiguous

True
False
tensor([[0.0801, 0.7271, 0.3827],
        [0.2099, 0.8101, 0.2096]])


### Relationship between `.view()` and `.flatten()`

` x.view(-1) == x.flatten() `

The only difference:
- `.flatten()` can flatten across specific dimensions (start_dim, end_dim)
- `.view(-1)` flattens the entire tensor.


In [33]:
a = torch.rand((3,4,5,6,7)) # shape = (3,4,5,6,7)
b = a.flatten(start_dim=2) # shape = (3,4,210)
print(b.shape) # Flattens starting from the start_dim

c = a.view(-1) # flattens along all dimensions
print(c.shape) # [2520,]

torch.Size([3, 4, 210])
torch.Size([2520])


### When .view() fails silently (common trap)
- You may think .view() reshaped the tensor correctly, but if the tensor was non-contiguous, and you used .contiguous() incorrectly, you might accidentally copy data unknowingly.

In [35]:
x = torch.randn(2, 3, 4)
y = x.permute(0, 2, 1)          # non-contiguous
z = y.contiguous().view(2, -1)  # makes a copy

# Z no longer shares memory with y